In [1]:
import re
import pandas as pd
from pprint import pprint
import numpy as np
import itertools

In [2]:
def pctTable(resampleData):
    pctList = []
    valuesnum = len([1 for key in resampleData.columns if "val" in key])
    for idx in range(valuesnum):
        sample = pd.Series([0.0] * 31, index=list(range(1, 31+1)))
        valCol = resampleData.values[:, idx].astype(int)
        for v in valCol:
            sample[v] += 1
        sample /= len(valCol)
        pctList.append(sample)
    pctList = np.vstack(pctList).T
    dfpct = pd.DataFrame(pctList, index=sample.index)
    return dfpct

In [3]:
def pctRank(data, num: int=2):
    s = data.sort_values()
    s = s.drop(s[s==0].index)
    rank = []
    count = 0
    index = 0
    while count != num:
        try:
            flag = s.duplicated()[::-1].iloc[index]
            if not flag:
                count += 1
            rank.append(s[::-1].index[index])
            index += 1
        except:
            break
    return rank

In [4]:
def CheckResult(data):
    _, data = data
    dfpct = pctTable(data)
    rank = list(map(pctRank, [dfpct[i] for i in range(5)]))
    kumi = list(map(list, itertools.product(*rank)))
    count = 0
    for i in range(len(kumi)):
        fmt = " & ".join([f"val{idx+1}=={val}" for idx, val in enumerate(kumi[i])])
        if len(data.query(fmt)) == 1:
            count += 1
    pct = count / len(data)
    return len(kumi), pct

In [5]:
def main(sampling: str="A", rank: int=2):
    chikan = re.compile("[年月日]")
    df = pd.read_csv("miniloto.csv")
    df["date"] = pd.to_datetime(df["date"].map(lambda x: str(x).replace("年", "-").replace("月", "-").replace("日", "-")))
    df = df.drop_duplicates("date")
    df = df.set_index("date")
    df = df.dropna()
    resample = df.copy()
    resample = resample.drop("count", axis=1)
    resample = resample.drop("bornas", axis=1)
    resample = resample.astype(int)
    Colums = len(resample.keys())
    resample = list(resample.resample(sampling))
    T = resample[:-1]
    T_1 = resample[1:]
    print("Start Analysis")
    for idx in range(len(T)):
        try:
            t = T[idx]
            t_1 = T_1[idx]
            DateTime, Values = t
            DateTime1, Values1 = t_1
            dfpct = pctTable(Values)
            rank = list(map(lambda x: pctRank(x, rank), [dfpct[i] for i in range(Colums)]))
            kumi = list(map(list, itertools.product(*rank)))
            count = 0
            print("Check PCT Start: ", len(kumi))
            for i in range(len(kumi)):
                fmt = " & ".join([f"val{idx+1}=={val}" for idx, val in enumerate(kumi[i])])
                if len(Values1.query(fmt)) == 1:
                    count += 1
                print(f"Run... {i+1}/{len(kumi)}\r", end="")
            pct = count / len(Values1)
            print(DateTime1, len(kumi), pct)
            print("Check PCT END: ", f"{idx + 1} / {len(T)}")
            print("-" * 50)
        except:
            pass
    print("END Analysis")

In [1]:
200 * 134

26800